In [2]:
import sys
sys.path.append("../")

import pandas as pd
import os
import statsmodels.formula.api as sm
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from __future__ import print_function
from patsy import dmatrices
from openpyxl import load_workbook

from library import regulations
from library import characteristics
from library import analysis
from library import tables

In [5]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/impact/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'cits_match.csv'),
                  sep=",", low_memory = False)
data.head()
data = data[data.year == 2016]
data['doi_16'] = np.where(data.doi_year == 2016, 1, 0)
data['doi_17'] = np.where(data.doi_year == 2017, 1, 0)
data['doi_18'] = np.where(data.doi_year == 2018, 1, 0)
data['tpsd'] = np.where(data.doi == False, 1, 0)
data = data[pd.notnull(data.psweight)]
data.head()

,Unnamed: 0,Unnamed: 0.1,district,distname,year,cntyname,distischarter,rating_academic,rating_financial,eligible,...,scores13,scores14,scores15,ps,psweight,pswtshare,doi_16,doi_17,doi_18,tpsd
4,4,4901,1902,CAYUGA ISD,2016,ANDERSON,N,M,Pass,True,...,0.687344,0.698003,0.766679,0.857158,1.166646,0.134851,0,1,0,0
11,11,4902,1903,ELKHART ISD,2016,ANDERSON,N,M,Pass,True,...,0.127778,0.322875,0.552578,0.911976,1.096520,0.126745,0,0,1,0
18,18,4903,1904,FRANKSTON ISD,2016,ANDERSON,N,M,Pass,True,...,0.201075,0.257979,0.530431,0.833157,1.200254,0.138736,0,0,1,0
25,25,4904,1906,NECHES ISD,2016,ANDERSON,N,M,Pass,True,...,0.873445,0.883988,0.274774,0.838313,1.192872,0.137883,0,1,0,0
32,32,4905,1907,PALESTINE ISD,2016,ANDERSON,N,M,Pass,True,...,-0.538253,-0.363930,-0.255851,0.836509,1.195444,0.138180,1,0,0,0


# Make mini table

In [6]:
def coef_with_stars(coef, pvalue):
    if pvalue >.05:
        coef = str(coef)
    if pvalue <= .05:
        coef = str(coef) + '*'
    if pvalue <= .01:
        coef = coef + '*'
    if pvalue <= .001:
        coef = coef + '*'
    return(coef)
test = coef_with_stars(.1, .005)
test

'0.1**'

In [18]:
data = data[pd.notnull(data.psweight)]
formula = 'students_num' + ' ~ ' + 'doi + doi_16 + doi_18'
y, X = dmatrices(formula, data=data, return_type='dataframe', NA_action = "drop") 
psweights = data.psweight
#mod = smf.WLS(y, X, weights = psweights)    # Describe model
#res = mod.fit()       # Fit model
len(psweights)

902

In [20]:
def make_coef_df(data, y_list, y_labels):
    regs = []
    cons = []
    coef_1 = []
    coef_2 = []
    coef_3 = []

    for var in y_list:
        formula = var + ' ~ ' + 'doi + doi_16 + doi_18'
        y, X = dmatrices(formula, data=data, return_type='dataframe', NA_action = "drop") 
        psweights = data.psweight
        mod = smf.WLS(y, X, weights = psweights)    # Describe model
        res = mod.fit()       # Fit model
        print(y_labels[var])
        print(res.summary())   # Summarize model
        cons.append(str(res.params["Intercept"].round(2)))
        cons.append('')
        coef_1.append(coef_with_stars(res.params['doi[T.True]'].round(2), res.pvalues['doi[T.True]']))
        coef_1.append('(' + str(res.bse['doi[T.True]'].round(2)) + ')')
        coef_2.append(coef_with_stars(res.params['doi_16'].round(2), res.pvalues['doi_16']))
        coef_2.append('(' + str(res.bse['doi_16'].round(2)) + ')')
        coef_3.append(coef_with_stars(res.params['doi_18'].round(2), res.pvalues['doi_18']))
        coef_3.append('(' + str(res.bse['doi_18'].round(2)) + ')')
        regs.append(y_labels[var])
        regs.append('')


    df = pd.DataFrame(
        {'Characteristic': regs,
         'Control': cons,
         'DOI Difference': coef_1,
         'Early Adopter': coef_2,
         'Late Adopter': coef_3,
         })
    return df
geo = make_coef_df(data = data, 
                                 y_list = characteristics.geography,
                                 y_labels = characteristics.labels)
geo

Urban
                            WLS Regression Results                            
Dep. Variable:             type_urban   R-squared:                       0.008
Model:                            WLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     2.324
Date:                Sun, 17 Mar 2019   Prob (F-statistic):             0.0736
Time:                        14:32:35   Log-Likelihood:                 3.1692
No. Observations:                 902   AIC:                             1.662
Df Residuals:                     898   BIC:                             20.88
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0500      0.010      4.884

,Characteristic,Control,DOI Difference,Early Adopter,Late Adopter
0,Urban,0.05,-0.01,0.09*,0.05
1,,,(0.02),(0.04),(0.03)
2,Suburban,0.22,-0.01,0.12,0.02
3,,,(0.03),(0.07),(0.06)
4,Town,0.34,-0.04,-0.1,0.0
5,,,(0.03),(0.08),(0.06)
6,Rural,0.39,0.06,-0.11,-0.06
7,,,(0.03),(0.09),(0.07)


In [21]:
teacher = make_coef_df(data = data, 
                                 y_list = characteristics.teacher,
                                 y_labels = characteristics.labels)
teacher

Ave. Experience Teaching
                             WLS Regression Results                            
Dep. Variable:     teachers_tenure_ave   R-squared:                       0.003
Model:                             WLS   Adj. R-squared:                 -0.000
Method:                  Least Squares   F-statistic:                    0.8833
Date:                 Sun, 17 Mar 2019   Prob (F-statistic):              0.449
Time:                         14:33:35   Log-Likelihood:                -1935.6
No. Observations:                  902   AIC:                             3879.
Df Residuals:                      898   BIC:                             3898.
Df Model:                            3                                         
Covariance Type:             nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      

,Characteristic,Control,DOI Difference,Early Adopter,Late Adopter
0,Ave. Experience Teaching,7.36,-0.21,0.16,0.08
1,,,(0.13),(0.33),(0.26)
2,Teacher Turnover Ratio,0.19,-0.0,-0.01,-0.0
3,,,(0.01),(0.02),(0.01)
4,Student-Teacher Ratio,6.59,-0.0,0.31,0.23
5,,,(0.08),(0.21),(0.16)


In [24]:
student = make_coef_df(data = data[pd.notnull(data.avescores)], 
                                 y_list = characteristics.student,
                                 y_labels = characteristics.labels)
student

Percent Hispanic
                            WLS Regression Results                            
Dep. Variable:          students_hisp   R-squared:                       0.009
Model:                            WLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     2.691
Date:                Sun, 17 Mar 2019   Prob (F-statistic):             0.0452
Time:                        14:34:19   Log-Likelihood:                -177.41
No. Observations:                 901   AIC:                             362.8
Df Residuals:                     897   BIC:                             382.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.3607      0.013

,Characteristic,Control,DOI Difference,Early Adopter,Late Adopter
0,Percent Hispanic,0.36,0.03,0.09,0.01
1,,,(0.02),(0.05),(0.04)
2,Percent White,0.54,-0.03,-0.1*,-0.01
3,,,(0.02),(0.05),(0.04)
4,Percent Black,0.06,0.0,-0.01,-0.0
5,,,(0.01),(0.02),(0.01)
6,Percent Econ. Disadvantaged,0.57,0.0,0.01,-0.01
7,,,(0.01),(0.03),(0.03)
8,Average STAAR Performance (Std.),0.31,-0.0,0.1,0.03
9,,,(0.05),(0.12),(0.1)


# To Table

In [28]:
rows = [6, 15, 22]

In [32]:
dfs = [geo, teacher, student]
file = table_path + 'regression_doi_year.xlsx'
wb = load_workbook(file)
ws = wb.active
str_n = 'N = ' + str(len(data[data.doi == False]))
ws.cell(row=4, column=2).value = str_n
str_n = 'N = ' + str(len(data[data.doi == True]))

for df, row in zip(dfs, rows):
    ws.cell(row=4, column=3).value = str_n
    row_n = row
    for ob in df.Control:
        ws.cell(row=row_n, column=2).value = ob
        row_n = row_n + 1
    row_n = row
    for ob in df['DOI Difference']:
        ws.cell(row=row_n, column=3).value = ob
        row_n = row_n + 1   
    row_n = row
    for ob in df['Early Adopter']:
        ws.cell(row=row_n, column=4).value = ob
        row_n = row_n + 1 
    row_n = row
    for ob in df['Late Adopter']:
        ws.cell(row=row_n, column=5).value = ob
        row_n = row_n + 1 
wb.save(file)